In [ ]:
#!/usr/bin/env python3

from ortools.sat.python import cp_model

def main():
    num_profesors = 5
    num_turnos = 2
    num_dias = 4
    all_profesors = range(num_profesors)
    all_turnos = range(num_turnos)
    all_dias = range(num_dias)
    turno_pref = [[[0, 0], [0, 0], [0, 0], [0, 0]],
                      [[0, 0], [1, 1], [1, 1], [1, 1]],
                      [[0, 0], [0, 0], [0, 0], [0, 0]],
                      [[1, 0], [1, 0], [1, 0], [1, 0]],
                      [[0, 1], [0, 1], [0, 1], [0, 1]]]

    model = cp_model.CpModel()

    turnos = {}
    for n in all_profesors:
        for d in all_dias:
            for s in all_turnos:
                turnos[(n, d,
                        s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

    for d in all_dias:
        for s in all_turnos:
            model.Add(sum(turnos[(n, d, s)] for n in all_profesors) == 1)

    for n in all_profesors:
        for d in all_dias:
            model.Add(sum(turnos[(n, d, s)] for s in all_turnos) <= 1)

    min_turnos_para_profesor = (num_turnos * num_dias) // num_profesors
    if num_turnos * num_dias % num_profesors == 0:
        max_turnos_para_profesor = min_turnos_para_profesor
    else:
        max_turnos_para_profesor = min_turnos_para_profesor + 1
    for n in all_profesors:
        num_turnos_trabaja = 0
        for d in all_dias:
            for s in all_turnos:
                num_turnos_trabaja += turnos[(n, d, s)]
        model.Add(min_turnos_para_profesor <= num_turnos_trabaja)
        model.Add(num_turnos_trabaja <= max_turnos_para_profesor)

    model.Maximize(
        sum(turno_pref[n][d][s] * turnos[(n, d, s)] for n in all_profesors
            for d in all_dias for s in all_turnos))

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        print('\nSolucion de asignación de turnos (clases) a profesores UTAD:\n')
        for d in all_dias:
            if d == 0:
                print('[Lunes]:\n')
            if d == 1:
                print('[Martes]:\n')
            if d == 2:
                print('[Miercoles]:\n')
            if d == 3:
                print('[Jueves]:\n')
            for n in all_profesors:
                for s in all_turnos:
                    if solver.Value(turnos[(n, d, s)]) == 1:
                        if turno_pref[n][d][s] == 1:
                            if n == 0:
                                print('Profesore', 'Mar', 'trabaja en el turno', "de mañana. (asignación con preferencias)" if s == 0 else "de tarde. (asignación con preferencias)")
                            if n == 1:
                                print('Profesore', 'Ramona', 'trabaja en el turno', "de mañana. (asignación con preferencias)" if s == 0 else "de tarde. (asignación con preferencias)")
                            if n == 2:
                                print('Profesore', 'Teresa', 'trabaja en el turno', "de mañana. (asignación con preferencias)" if s == 0 else "de tarde. (asignación con preferencias)")
                            if n == 3:
                                print('Profesore', 'Manoel', 'trabaja en el turno', "de mañana. (asignación con preferencias)" if s == 0 else "de tarde. (asignación con preferencias)")
                            if n == 4:
                                print('Profesore', 'Pedro', 'trabaja en el turno', "de mañana. (asignación con preferencias)" if s == 0 else "de tarde. (asignación con preferencias)")
                        else:
                            if n == 0:
                                print('Profesore', 'Mar', 'trabaja en el turno', "de mañana. (asignación sin preferencias)" if s == 0 else "de tarde. (asignación sin preferencias)")
                            if n == 1:
                                print('Profesore', 'Ramona', 'trabaja en el turno', "de mañana. (asignación sin preferencias)" if s == 0 else "de tarde. (asignación sin preferencias)")
                            if n == 2:
                                print('Profesore', 'Teresa', 'trabaja en el turno', "de mañana. (asignación sin preferencias)" if s == 0 else "de tarde. (asignación sin preferencias)")
                            if n == 3:
                                print('Profesore', 'Manoel', 'trabaja en el turno', "de mañana. (asignación sin preferencias)" if s == 0 else "de tarde. (asignación sin preferencias)")
                            if n == 4:
                                print('Profesore', 'Pedro', 'trabaja en el turno', "de mañana. (asignación sin preferencias)" if s == 0 else "de tarde. (asignación sin preferencias)")

            print()
            
        print(f'Número de solicitudes de turno atendidas = {solver.ObjectiveValue()}',
              f'(fuera de {num_profesors * min_turnos_para_profesor})')
    else:
        print('¡No se encontró una solución óptima!')

    print('\nEstadísticas')
    print('  - conflictos: %i' % solver.NumConflicts())
    print('  - branches : %i' % solver.NumBranches())
    print('  - tiempo de pared: %f s' % solver.WallTime())
if __name__ == '__main__':
    main()

In [ ]:
#!/usr/bin/env python3


from ortools.sat.python import cp_model


class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count


def CovidLover():

    model = cp_model.CpModel()

    base = 12

    c = model.NewIntVar(1, base - 1, 'C')
    o = model.NewIntVar(0, base - 1, 'O')
    v = model.NewIntVar(0, base - 1, 'V')
    i = model.NewIntVar(0, base - 1, 'I')
    d = model.NewIntVar(0, base - 1, 'D')
    l = model.NewIntVar(1, base - 1, 'L')
    r = model.NewIntVar(0, base - 1, 'R')
    e = model.NewIntVar(0, base - 1, 'E')
    s = model.NewIntVar(0, base - 1, 'S')

    letters = [c, o, v, i, d, l, r, e, s]

    assert base >= len(letters)

    model.AddAllDifferent(letters)

    model.Add((c * base**4 + o * base**3 + v * base**2 + i * base + d) +
                (l * base**4 + o * base**3 + v * base**2 + e * base + r) ==
                (d * base**5 + i * base**4 + v * base**3 + i * base**2 + d * base + e))

    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinter(letters)

    solver.parameters.enumerate_all_solutions = True

    status = solver.Solve(model, solution_printer)

    print()
    print('Statistics')
    print('  - status          : %s' % solver.StatusName(status))
    print('  - conflicts       : %i' % solver.NumConflicts())
    print('  - branches        : %i' % solver.NumBranches())
    print('  - wall time       : %f s' % solver.WallTime())
    print('  - solutions found : %i' % solution_printer.solution_count())


if __name__ == '__main__':
    CovidLover()
